In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hemul import heaan
import hemul.HEAAN as he
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import load_params, load_img
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv
from muxcnn.utils import get_channel_last, get_conv_params
from muxcnn.hecnn_par import MultParPack
from muxcnn.hecnn_par import forward_convbn_par

## Torch model

In [3]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

img_tensor = load_img("./cute.jpg", hi=32, wi=32)

## Image

In [4]:
imgl = get_channel_last(img_tensor[0].detach().numpy())
ki = 1 # initial ki
hi, wi, ch = imgl.shape

# FHE

In [5]:
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

# try 40 & 800
hec = heaan.HEAANContext(15, 40, 800, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
HEAAN CKKS setup is ready 


In [6]:
fhemodel = ResNetHEAAN(model, hec)

ctxt = fhemodel.pack_img_ctxt(img_tensor)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


# forward early

In [7]:
ctxt0, outs0 = fhemodel.forward_early(ctxt, 1, 32, 32)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
ct_a 40 800 [-7.49019623e-01 -9.13725499e-01 -5.52941204e-01 -8.43137266e-01
 -9.40567289e-09 -4.11764679e-01  9.80392721e-02 -8.50980403e-01
 -1.26314386e-10 -2.47058828e-01  1.29411808e-01 -6.78431388e-01
  4.14758520e-10  8.23529968e-02  2.94117690e-01  2.31372594e-01
 -4.66066948e-09  1.92156913e-01  1.52941228e-01  1.29411814e-01
 -1.82949109e-09  2.78431416e-01 -5.52941200e-01 -5.45098064e-01
 -1.20780713e-09  1.05882407e-01  4.98039249e-01  4.50980425e-01
 -1.02603173e-10  7.41176486e-01  7.41176484e-01 -4.27450951e-01
  2.55702095e-09]
ct_d 40 720
40 760
40 760
Check 1
[ 1.23083784  0.48697924  0.27534382 ... -0.65056937 -0.61289192
 -0.47934082]
APPR 40 720
[ 1.23083784  0.48697924  0.27534382 ... -0.65056937 -0.61289192
 -0.47934082]
min max -5.745771389394943 5.543758011739413
APPR 40 520
[ 0.60359733  0.25016229  0.14227706 ... -0.3319452  -0.3

In [8]:
dec = hec.decrypt(ctxt0)
print(dec[::1000])

[ 1.23083789e+00  1.37002640e-09  9.00872489e-01 -4.38359752e-10
  1.03879856e+00 -1.65514497e-03  1.72884406e-02  2.26538539e-01
 -2.33773276e-09  4.19786160e-09  3.50234374e-09 -7.55062574e-09
  5.43471055e-09  4.46616520e-01 -5.76709296e-10  1.06025070e+00
 -3.20269816e-08  4.43068578e-10  2.69868618e-01  3.97408014e+00
  1.03376660e+00  1.13955701e-09  5.28608122e-01  2.52203229e+00
  3.99438942e-09  8.40950253e-02  1.34547451e-09  4.63119015e-09
  1.73388720e-08  6.15333969e-01  5.37446504e-01  4.76048129e-01
  7.81339991e-09]


In [9]:
hec.rescale(ctxt0)

In [ ]:
ctxt1, outs1 = fhemodel.forward_bb(model.layer1[0], ctxt0, outs0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
ct_a 40 520 [ 1.23083789e+00  1.30622336e-09  9.00872489e-01 -3.98414064e-10
  1.03879856e+00 -1.65514445e-03  1.72884410e-02  2.26538539e-01
 -2.40784584e-09  4.79207252e-09  4.06273613e-09 -7.62289848e-09
  5.12659448e-09  4.46616520e-01 -6.98286823e-10  1.06025070e+00
 -3.17134207e-08  4.05157479e-10  2.69868618e-01  3.97408014e+00
  1.03376660e+00  1.25584526e-09  5.28608122e-01  2.52203229e+00
  3.91535067e-09  8.40950255e-02  1.44473885e-09  4.58060144e-09
  1.71692909e-08  6.15333969e-01  5.37446504e-01  4.76048127e-01
  8.21522409e-09]
ct_d 40 440
40 480
40 480
40 480
40 480
40 480
40 480
40 480
40 480
APPR 40 440
[ 0.86425903  1.07560652  0.71689016 ...  0.7777809   1.14699243
 -0.12684551]
min max -7.502171073600742 4.664819357483846
APPR 40 240
[ 0.43581932  0.53437552  0.36458054 ...  0.39424057  0.56656664
 -0.06568679]
min max -1.177562745141

In [ ]:
ctxt2, outs2 = fhemodel.forward_bb(model.layer2[0], ctxt1, outs1)

In [ ]:
ctxt3, outs3 = fhemodel.forward_bb(model.layer3[0], ctxt2, outs2)

In [5]:
# early conv and bn
_, ins0, outs0 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins0)

In [6]:
# Muxed
U, ins1, outs1 = get_conv_params(model.conv1, {'k':ki, 'h':hi, 'w':wi})
ct_a = MultParPack(imgl, ins1)

out1, un1 = forward_convbn_par(model.conv1, 
                              model.bn1, ct_a, ins1)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.7468 s


## Activation